In [1]:
import requests
import re
import pandas as pd
import json
from bs4 import BeautifulSoup
from googleapiclient.discovery import build

In [212]:
def save_json(loc, data, mode = 'w'):
    with open(loc, mode) as outfile:
        json.dump(data, outfile)
        outfile.close()
    return True

In [2]:
from keys import api_key

In [3]:
youtubers = {
    'krishnaik':
    {
        'username': 'krishnaik06'
    },
    "Hitesh":
    {
        'username': 'HiteshChoudharydotcom'
    },
    
    'mysirji': 
    {
        'username': 'saurabhexponent1'
    },
    
    'Telusko':
    {
        'username': 'javaboynavin'
    }
}

In [238]:
for youtuber in youtubers.keys():
    print(youtuber)
    url = "https://www.youtube.com/user/" + youtubers[youtuber]['username'] + "/videos"
    print(url)

krishnaik
https://www.youtube.com/user/krishnaik06/videos
Hitesh
https://www.youtube.com/user/HiteshChoudharydotcom/videos
mysirji
https://www.youtube.com/user/saurabhexponent1/videos
Telusko
https://www.youtube.com/user/javaboynavin/videos


# Getting channelids

In [4]:
for youtuber in youtubers.keys():
    print(youtuber)
    url = "https://www.youtube.com/user/" + youtubers[youtuber]['username'] + "/videos"
    response = requests.get(url)
    
    if response.ok == True:
        doc = BeautifulSoup(response.text, 'html.parser')
    else:
        url = "https://www.youtube.com/c/" + youtubers[youtuber]['username'] + "/videos"
        response = requests.get(url)
        doc = BeautifulSoup(response.text, 'html.parser')
        
    channelId = doc.find('link', rel = re.compile("canonical"))['href'].split('/')[-1]
    youtubers[youtuber]['channelId'] = channelId

krishnaik
Hitesh
mysirji
Telusko


In [5]:
pd.DataFrame(youtubers).T

,username,channelId
krishnaik,krishnaik06,UCNU_lfiiWBdtULKOw6X0Dig
Hitesh,HiteshChoudharydotcom,UCXgGY0wkgOzynnHvSEVmE3A
mysirji,saurabhexponent1,UCkGS_3D0HEzfflFnG0bD24A
Telusko,javaboynavin,UC59K-uG2A5ogwIrHw4bmlEg


In [193]:
save_json('youtubers.json', youtubers)

True

# Getting playlistIds

In [241]:
youtube_build = build('youtube', 'v3', developerKey = api_key)

In [242]:
for youtuber in youtubers.keys():
    print(youtuber)
    channelId = youtubers[youtuber]['channelId']
    response = youtube_build.channels().list(
                            part='contentDetails',
                            id = channelId
                                    ).execute()
    playlistId = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    youtubers[youtuber]['playlistId'] = playlistId

krishnaik
Hitesh
mysirji
Telusko


In [243]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'l-cA7n0Gv_LMxW-iQbD57ZF_mMs',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'CTaliWoPV0JfVG4ToBpGUMM9OrU',
   'id': 'UC59K-uG2A5ogwIrHw4bmlEg',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UU59K-uG2A5ogwIrHw4bmlEg'}}}]}

In [8]:
pd.DataFrame(youtubers).T

,username,channelId,playlistId
krishnaik,krishnaik06,UCNU_lfiiWBdtULKOw6X0Dig,UUNU_lfiiWBdtULKOw6X0Dig
Hitesh,HiteshChoudharydotcom,UCXgGY0wkgOzynnHvSEVmE3A,UUXgGY0wkgOzynnHvSEVmE3A
mysirji,saurabhexponent1,UCkGS_3D0HEzfflFnG0bD24A,UUkGS_3D0HEzfflFnG0bD24A
Telusko,javaboynavin,UC59K-uG2A5ogwIrHw4bmlEg,UU59K-uG2A5ogwIrHw4bmlEg


# Getting
# 1. video title
# 2. publish time
# 3. thumnail Link
# 4. video Link
# 5. video Id

In [194]:
data = {
    'youtuber':[],
    'username': [],
    'channelTitle': [],
    'title': [],
    'publishedAt': [],
    'thumbnailLink': [],
    'videoLink': [],
    'videoId':[]
}



for youtuber in youtubers.keys():
    print(youtuber)
    playlistId = youtubers[youtuber]['playlistId']
    response_playlist = youtube_build.playlistItems().list(
                                part = "snippet",
                                playlistId = playlistId,
                                maxResults=50
                                ).execute()
    for item in response_playlist['items']:
        snippet = item['snippet']
        data['youtuber'].append(youtuber)
        data['username'].append(youtubers[youtuber]['username'])
        data['channelTitle'].append(snippet['channelTitle'])
        data['title'].append(snippet['title'])
        data['publishedAt'].append(snippet['publishedAt'])
        data['thumbnailLink'].append(snippet['thumbnails']['default']['url'])
        videoId = snippet['resourceId']['videoId']
        data['videoId'].append(videoId)
        videoLink = f"https://www.youtube.com/watch?v={videoId}"
        data['videoLink'].append(videoLink)

krishnaik
Hitesh
mysirji
Telusko


In [195]:
pd.DataFrame(data)

,youtuber,username,channelTitle,title,publishedAt,thumbnailLink,videoLink,videoId
0,krishnaik,krishnaik06,Krish Naik,Hyperparameter Tuning Using Machine Learning P...,2022-09-03T11:20:06Z,https://i.ytimg.com/vi/82fPl5l0vXY/default.jpg,https://www.youtube.com/watch?v=82fPl5l0vXY,82fPl5l0vXY
1,krishnaik,krishnaik06,Krish Naik,Solve this Data Science Interview Question Fro...,2022-09-02T15:13:14Z,https://i.ytimg.com/vi/bDJkMOvhAmc/default.jpg,https://www.youtube.com/watch?v=bDJkMOvhAmc,bDJkMOvhAmc
2,krishnaik,krishnaik06,Krish Naik,"FAANG Interview DSA Series-Heap Sort,Binary Tr...",2022-09-02T11:11:47Z,https://i.ytimg.com/vi/AxALCZMo01Y/default.jpg,https://www.youtube.com/watch?v=AxALCZMo01Y,AxALCZMo01Y
3,krishnaik,krishnaik06,Krish Naik,Implementing Machine Learninng Pipelines USsin...,2022-09-01T13:52:19Z,https://i.ytimg.com/vi/HZ9MUzCRlzI/default.jpg,https://www.youtube.com/watch?v=HZ9MUzCRlzI,HZ9MUzCRlzI
4,krishnaik,krishnaik06,Krish Naik,FAANG DSA Series-Discussing Sorting Algorithms...,2022-08-31T10:38:45Z,https://i.ytimg.com/vi/h9pB1RaTa2U/default.jpg,https://www.youtube.com/watch?v=h9pB1RaTa2U,h9pB1RaTa2U
...,...,...,...,...,...,...,...,...
195,Telusko,javaboynavin,Telusko,Full Stack Blockchain Course | Affordable Price,2022-06-23T17:04:53Z,https://i.ytimg.com/vi/GDdigOQ4qOY/default.jpg,https://www.youtube.com/watch?v=GDdigOQ4qOY,GDdigOQ4qOY
196,Telusko,javaboynavin,Telusko,Java Live Session | Constructor and Encapsulation,2022-06-23T14:53:50Z,https://i.ytimg.com/vi/s72FvmjrTw4/default.jpg,https://www.youtube.com/watch?v=s72FvmjrTw4,s72FvmjrTw4
197,Telusko,javaboynavin,Telusko,SubQuery Project publish on SubQuery Network,2022-06-23T12:22:23Z,https://i.ytimg.com/vi/yit1hecSfJE/default.jpg,https://www.youtube.com/watch?v=yit1hecSfJE,yit1hecSfJE
198,Telusko,javaboynavin,Telusko,Hackathon 3.0 in collab with MuSigma | AI/ML,2022-06-23T02:33:43Z,https://i.ytimg.com/vi/Yrl2oHO1Ywk/default.jpg,https://www.youtube.com/watch?v=Yrl2oHO1Ywk,Yrl2oHO1Ywk


# Getting
# 1. likes
# 2. no of comments
# 3. comments with commentor's name

In [197]:
data_temp = {
    'viewCount': [],
    'likeCount': [],
    'commentCount': []
}

for videoId in data['videoId']:
    response_video = youtube_build.videos().list(
                            part = "statistics",
                            id = videoId
                        ).execute()
    data_temp['viewCount'].append(response_video['items'][0]['statistics']['viewCount'])
    data_temp['likeCount'].append(response_video['items'][0]['statistics']['likeCount'])
    data_temp['commentCount'].append(response_video['items'][0]['statistics']['commentCount'])

In [198]:
data = {**data, **data_temp}

In [199]:
pd.DataFrame(data)

,youtuber,username,channelTitle,title,publishedAt,thumbnailLink,videoLink,videoId,viewCount,likeCount,commentCount
0,krishnaik,krishnaik06,Krish Naik,Hyperparameter Tuning Using Machine Learning P...,2022-09-03T11:20:06Z,https://i.ytimg.com/vi/82fPl5l0vXY/default.jpg,https://www.youtube.com/watch?v=82fPl5l0vXY,82fPl5l0vXY,1528,117,8
1,krishnaik,krishnaik06,Krish Naik,Solve this Data Science Interview Question Fro...,2022-09-02T15:13:14Z,https://i.ytimg.com/vi/bDJkMOvhAmc/default.jpg,https://www.youtube.com/watch?v=bDJkMOvhAmc,bDJkMOvhAmc,6700,215,16
2,krishnaik,krishnaik06,Krish Naik,"FAANG Interview DSA Series-Heap Sort,Binary Tr...",2022-09-02T11:11:47Z,https://i.ytimg.com/vi/AxALCZMo01Y/default.jpg,https://www.youtube.com/watch?v=AxALCZMo01Y,AxALCZMo01Y,2687,104,13
3,krishnaik,krishnaik06,Krish Naik,Implementing Machine Learninng Pipelines USsin...,2022-09-01T13:52:19Z,https://i.ytimg.com/vi/HZ9MUzCRlzI/default.jpg,https://www.youtube.com/watch?v=HZ9MUzCRlzI,HZ9MUzCRlzI,9005,381,29
4,krishnaik,krishnaik06,Krish Naik,FAANG DSA Series-Discussing Sorting Algorithms...,2022-08-31T10:38:45Z,https://i.ytimg.com/vi/h9pB1RaTa2U/default.jpg,https://www.youtube.com/watch?v=h9pB1RaTa2U,h9pB1RaTa2U,2561,86,10
...,...,...,...,...,...,...,...,...,...,...,...
195,Telusko,javaboynavin,Telusko,Full Stack Blockchain Course | Affordable Price,2022-06-23T17:04:53Z,https://i.ytimg.com/vi/GDdigOQ4qOY/default.jpg,https://www.youtube.com/watch?v=GDdigOQ4qOY,GDdigOQ4qOY,36446,492,139
196,Telusko,javaboynavin,Telusko,Java Live Session | Constructor and Encapsulation,2022-06-23T14:53:50Z,https://i.ytimg.com/vi/s72FvmjrTw4/default.jpg,https://www.youtube.com/watch?v=s72FvmjrTw4,s72FvmjrTw4,14138,764,18
197,Telusko,javaboynavin,Telusko,SubQuery Project publish on SubQuery Network,2022-06-23T12:22:23Z,https://i.ytimg.com/vi/yit1hecSfJE/default.jpg,https://www.youtube.com/watch?v=yit1hecSfJE,yit1hecSfJE,1207,32,5
198,Telusko,javaboynavin,Telusko,Hackathon 3.0 in collab with MuSigma | AI/ML,2022-06-23T02:33:43Z,https://i.ytimg.com/vi/Yrl2oHO1Ywk/default.jpg,https://www.youtube.com/watch?v=Yrl2oHO1Ywk,Yrl2oHO1Ywk,3444,123,4


In [216]:
pd.DataFrame(data).to_excel('data.xlsx')

In [200]:
save_json('data.json', data)

True

# Getting all comments, commentators and replies for all videos

In [110]:
def get_response_comments(videoId, nextPageToken = None):
    temp = []
    response_comments = youtube_build.commentThreads().list(
                            part='snippet,replies',
                            videoId = videoId,
                            pageToken = nextPageToken
                        ).execute()
    temp.extend(response_comments['items'])
    if 'nextPageToken' in response_comments:
        response = get_response_comments(videoId, nextPageToken = response_comments['nextPageToken'])
        if response:
            temp.extend(response)
    else:
        return temp
    return temp

In [227]:
data_comments = []
replies_keys = ['authorName', 'comment', 'likeCount', 'publishedAt']
replies_vals = ['authorDisplayName', 'textOriginal', 'likeCount', 'publishedAt']
for i, videoId in enumerate(data['videoId']):
    try:
        response_comments = get_response_comments(videoId)
        commentCount = len(response_comments)
        comments = {
                'videoId': videoId,
                'title': data['title'][i],
                'commentCount': commentCount,
                'comments': []
                   }
        for response in response_comments:
            totalReplyCount = response['snippet']['totalReplyCount']
            if totalReplyCount != 0:
                if 'replies' in response:
                    replies = [{k:val['snippet'][v] for k, v in zip(replies_keys, replies_vals)} 
                                for val in response['replies']['comments']]
                else:
                    print('No reply although totalReplyCount is ', totalReplyCount)
                    replies = []
            else:
                replies = []
            comments['comments'].append(
            {
                'authorName': response['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'comment': response['snippet']['topLevelComment']['snippet']['textOriginal'],
                'publishedAt': response['snippet']['topLevelComment']['snippet']['publishedAt'],
                'likeCount': response['snippet']['topLevelComment']['snippet']['likeCount'],
                'replies': replies
            })
    except Exception as e:
        print(e)
        print(response)
        comments = {
                'videoId': videoId,
                'commentCount': 0,
                'comments': []
                   }
    print(i, videoId, commentCount)
    data_comments.append(comments)

0 82fPl5l0vXY 11
1 bDJkMOvhAmc 13
2 AxALCZMo01Y 5
3 HZ9MUzCRlzI 30
4 h9pB1RaTa2U 10
5 MJ1vWb1rGwM 39
6 Pp6CO2_YEDE 14
7 -se8_ESCUTQ 10
8 qJJEf6p8xH0 23
9 VC2BL_ChGeg 23
10 C0KW4MvNbIQ 53
11 JneXvhlJW_k 260
12 uXEg8TF-2ZA 14
13 ITyGQpyMV_w 45
14 8-tyWaOFBPU 195
15 uwQYAH5MC8c 13
16 7uLzGRlXXDw 36
17 9di4_ZJqawU 33
18 gFwr2KxZ1pc 17
19 g9o4_ua2Gv0 29
20 015vL0cJfA0 21
21 ceJyWSgiiIA 19
22 gfvil_XC-kE 57
23 k2pofjdxkZ8 7
24 A3Xx8qaH8CQ 20
25 uqbIXNKL2OA 4
26 iyrEVohGQMM 16
27 r_NxVNbA034 38
28 ux1LX83ql_M 32
29 IaDS2y6hVUc 28
30 m_nQM_AkVfY 33
31 -rw3TtDCVRE 32
32 vnAN9pmgc2g 4
33 fXfs_6o_hOs 9
34 J_TLvwNYliM 12
35 Gs15V79cauo 23
36 SMqxisMGfYM 3
37 GW7B6vwktPA 18
38 Xii_iU9drSU 19
39 x_6EkOA3eNk 51
40 k3_qIfRogyY 22
41 o3cDrAXxcGE 15
42 8M1d8C5Dzug 10
43 bHcBzEkTZ1s 24
44 2_FqpMbKxf8 25
45 4-QDspUXS9A 29
46 WfmFoTKBw90 47
47 DIvvHPO1Tyo 69
<HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet%2Creplies&videoId=chkGtDtyGak&key=AIzaSyC09Ez4gCh

In [228]:
save_json('data_comments.json', data_comments)

True

In [244]:
data_comments[0]

{'videoId': '82fPl5l0vXY',
 'title': 'Hyperparameter Tuning Using Machine Learning Pipelines',
 'commentCount': 11,
 'comments': [{'authorName': 'Youtube Premium',
   'comment': "Hello Bhaiya, I have recently done my mbbs education.  And want to take admission in AI in medicine for my Master's. For the same, I need to learn ML and CNNs for medical image analysis using Deep Learning. \n\nI want to start from scratch.  How can I do the same? And any good universities you'd like to suggest to pursue my master's in AI medicine? ( open for foreign universities with scholarships as",
   'publishedAt': '2022-09-04T07:16:43Z',
   'likeCount': 0,
   'replies': []},
  {'authorName': 'Mayank Gupta',
   'comment': 'How can i train different models in single pipelining ?',
   'publishedAt': '2022-09-04T03:18:09Z',
   'likeCount': 0,
   'replies': []},
  {'authorName': 'MOGA MOGA',
   'comment': 'v=9oSHtBAN0mg thank  later lol',
   'publishedAt': '2022-09-04T02:44:06Z',
   'likeCount': 0,
   'replie

# Convert Thumbnail to base64

In [201]:
import base64

In [234]:
data_thumbnail_base64 = {
    'videoId': [],
    'title': [],
    'thumbnail': []
}

for thumbnailLink, videoId, title in zip(data['thumbnailLink'], data['videoId'], data['title']):
    response = requests.get(thumbnailLink)
    print(videoId, thumbnailLink)
    data_thumbnail_base64['videoId'].append(videoId)
    data_thumbnail_base64['title'].append(title)
    data_thumbnail_base64['thumbnail'].append(base64.b64encode(response.content))

82fPl5l0vXY https://i.ytimg.com/vi/82fPl5l0vXY/default.jpg
bDJkMOvhAmc https://i.ytimg.com/vi/bDJkMOvhAmc/default.jpg
AxALCZMo01Y https://i.ytimg.com/vi/AxALCZMo01Y/default.jpg
HZ9MUzCRlzI https://i.ytimg.com/vi/HZ9MUzCRlzI/default.jpg
h9pB1RaTa2U https://i.ytimg.com/vi/h9pB1RaTa2U/default.jpg
MJ1vWb1rGwM https://i.ytimg.com/vi/MJ1vWb1rGwM/default.jpg
Pp6CO2_YEDE https://i.ytimg.com/vi/Pp6CO2_YEDE/default.jpg
-se8_ESCUTQ https://i.ytimg.com/vi/-se8_ESCUTQ/default.jpg
qJJEf6p8xH0 https://i.ytimg.com/vi/qJJEf6p8xH0/default.jpg
VC2BL_ChGeg https://i.ytimg.com/vi/VC2BL_ChGeg/default.jpg
C0KW4MvNbIQ https://i.ytimg.com/vi/C0KW4MvNbIQ/default.jpg
JneXvhlJW_k https://i.ytimg.com/vi/JneXvhlJW_k/default.jpg
uXEg8TF-2ZA https://i.ytimg.com/vi/uXEg8TF-2ZA/default.jpg
ITyGQpyMV_w https://i.ytimg.com/vi/ITyGQpyMV_w/default.jpg
8-tyWaOFBPU https://i.ytimg.com/vi/8-tyWaOFBPU/default.jpg
uwQYAH5MC8c https://i.ytimg.com/vi/uwQYAH5MC8c/default.jpg
7uLzGRlXXDw https://i.ytimg.com/vi/7uLzGRlXXDw/default.j

Aadj7f_EdXA https://i.ytimg.com/vi/Aadj7f_EdXA/default.jpg
nyMORQAdhkk https://i.ytimg.com/vi/nyMORQAdhkk/default.jpg
-LUY_xSoj0M https://i.ytimg.com/vi/-LUY_xSoj0M/default.jpg
kjujkaEYryc https://i.ytimg.com/vi/kjujkaEYryc/default.jpg
AiXYFKAEugk https://i.ytimg.com/vi/AiXYFKAEugk/default.jpg
KKJTpmRTAsQ https://i.ytimg.com/vi/KKJTpmRTAsQ/default.jpg
cB__DlboSik https://i.ytimg.com/vi/cB__DlboSik/default.jpg
SQIJDy6xdx0 https://i.ytimg.com/vi/SQIJDy6xdx0/default.jpg
IGFeaXeQs6E https://i.ytimg.com/vi/IGFeaXeQs6E/default.jpg
Ir68iC9o_hM https://i.ytimg.com/vi/Ir68iC9o_hM/default.jpg
b7OryQDDCsU https://i.ytimg.com/vi/b7OryQDDCsU/default.jpg
3NfjY7ddHz8 https://i.ytimg.com/vi/3NfjY7ddHz8/default.jpg
4auwnxsEDeI https://i.ytimg.com/vi/4auwnxsEDeI/default.jpg
ujyvH3DjisE https://i.ytimg.com/vi/ujyvH3DjisE/default.jpg
UnffRY0btLQ https://i.ytimg.com/vi/UnffRY0btLQ/default.jpg
IC5vBKc21X8 https://i.ytimg.com/vi/IC5vBKc21X8/default.jpg
vTWpddJiUXI https://i.ytimg.com/vi/vTWpddJiUXI/default.j

In [235]:
pd.DataFrame(data_thumbnail_base64).to_csv('data_thumbnail_base64.csv')

In [218]:
############ END ##############